# CICIDS - Hybrid Model CNN-GAN

The CICIDS2017 dataset is a comprehensive dataset for network intrusion detection, created by the Canadian Institute for Cybersecurity. It includes a diverse set of attack scenarios and normal traffic, making it suitable for training and evaluating intrusion detection systems.

The dataset includes various types of attacks such as Brute Force, Heartbleed, Botnet, DoS (Denial of Service), DDoS (Distributed Denial of Service), Web attacks, and Infiltration of the network from inside.

I aim to replicate this study: <https://www.jait.us/articles/2024/JAIT-V15N7-886.pdf>

In [1]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## Step 1. EDA

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

In [3]:
df = pd.read_csv("../data/sampled/sampled_10000_10000.csv")

# Remove leading and trailing whitespaces from column names
df.columns = df.columns.str.strip()

In [4]:
df.head(5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,514645,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,53,339,2,2,62,748,31,31,31.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,39394,51,1,1,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,53,61777,1,1,45,151,45,45,45.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,53,23351,1,1,43,59,43,43,43.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [5]:
df.shape

(20000, 79)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 79 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Destination Port             20000 non-null  int64  
 1   Flow Duration                20000 non-null  int64  
 2   Total Fwd Packets            20000 non-null  int64  
 3   Total Backward Packets       20000 non-null  int64  
 4   Total Length of Fwd Packets  20000 non-null  int64  
 5   Total Length of Bwd Packets  20000 non-null  int64  
 6   Fwd Packet Length Max        20000 non-null  int64  
 7   Fwd Packet Length Min        20000 non-null  int64  
 8   Fwd Packet Length Mean       20000 non-null  float64
 9   Fwd Packet Length Std        20000 non-null  float64
 10  Bwd Packet Length Max        20000 non-null  int64  
 11  Bwd Packet Length Min        20000 non-null  int64  
 12  Bwd Packet Length Mean       20000 non-null  float64
 13  Bwd Packet Lengt

In [7]:
df.describe()

c:\Users\Harman\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\Harman\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,20000.000000,2.000000e+04,20000.000000,20000.000000,2.000000e+04,2.000000e+04,20000.000000,20000.00000,20000.000000,20000.000000,...,20000.000000,20000.000000,2.000000e+04,2.000000e+04,2.000000e+04,2.000000e+04,2.000000e+04,2.000000e+04,2.000000e+04,2.000000e+04
mean,5936.555950,1.658223e+07,9.299350,8.336800,1.429634e+03,8.780229e+03,197.491500,16.91885,57.821005,63.811307,...,4.719050,27.907200,3.232478e+05,1.138442e+05,4.313192e+05,2.444209e+05,8.488926e+06,1.036975e+06,9.639811e+06,7.677699e+06
std,15701.967114,3.329641e+07,102.350682,101.169224,3.591803e+04,2.007295e+05,787.346065,98.44432,222.373114,264.911494,...,78.839857,6.576317,1.467275e+06,7.633679e+05,1.824990e+06,1.355701e+06,2.352571e+07,5.304882e+06,2.492887e+07,2.318181e+07
min,0.000000,-1.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,53.000000,1.840000e+02,2.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.00000,0.000000,0.000000,...,0.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.000000,1.016825e+05,2.000000,2.000000,4.700000e+01,4.600000e+01,30.000000,0.00000,15.500000,0.000000,...,1.000000,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,443.000000,9.166827e+06,6.000000,4.000000,2.100000e+02,3.457500e+02,195.000000,6.00000,50.500000,67.803007,...,2.000000,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,65502.000000,1.199972e+08,5523.000000,6509.000000,2.866110e+06,1.160000e+07,23360.000000,1983.00000,5939.285714,5762.073497,...,5522.000000,56.000000,7.240000e+07,2.610000e+07,7.240000e+07,7.240000e+07,1.199200e+08,6.650000e+07,1.199200e+08,1.199200e+08


In [8]:
df.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

## Step 2. Data Cleaning

### A. Missing values

In [9]:
print(df.isna().sum().sum())

6


In [10]:
df.dropna(subset=["Flow Bytes/s"], inplace=True)

In [11]:
print(df.isna().sum().sum())

0


### Inf. values

In [12]:
df = df.replace([np.inf, -np.inf], np.nan).dropna()

## Step 3. Data Preparation

### A. Normalise numeric features

In [13]:
# Get all numerical columns
numerical_columns = df.select_dtypes(include="number").columns

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

### B. Map Labels to binary

In [15]:
# Change values in the column "Label" to 0 if BENIGN and 1 if not
df["Label"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)

df["Label"].value_counts()

Label
0    9994
1    9989
Name: count, dtype: int64

### C. Data Splitting

In [16]:
X = df.drop(columns="Label")
y = df["Label"]

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### E. PyTorch Model Design

#### 1. CNN Feature Extractor

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

# Define CNN Feature Extractor
class CNNFeatureExtractor(nn.Module):
    def __init__(self, input_size, num_filters=32):
        super(CNNFeatureExtractor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear((input_size // 2) * num_filters, 64)
    
    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        return self.fc(x)

#### 2. Generator-Discriminator

In [19]:
# Define Generator
class Generator(nn.Module):
    def __init__(self, noise_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.model(x)


# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

#### 3. Define Hybrid Model

In [20]:
# Define Hybrid Model
class HybridCNNGAN(nn.Module):
    def __init__(self, input_size, output_size, noise_dim=32):
        super(HybridCNNGAN, self).__init__()
        self.feature_extractor = CNNFeatureExtractor(input_size)
        self.classifier = nn.Linear(64, output_size)
        self.generator = Generator(noise_dim, input_size)
        self.discriminator = Discriminator(input_size)
    
    def forward(self, x):
        features = self.feature_extractor(x)
        return self.classifier(features)

In [21]:
# Initialize model
input_size = X_train.shape[1]
output_size = len(y_train.unique())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridCNNGAN(input_size, output_size).to(device)

### F. Training the Model

In [ ]:
import sys

# Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_dataset = TensorDataset(torch.tensor(X_train.values, dtype=torch.float32).to(device),
                              torch.tensor(y_train.values, dtype=torch.long).to(device))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Training Loop
epoch = 0
current_highest_accuracy = 0.0
patience = 50
epochs_without_improvement = 0

while True:
    epoch += 1
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for i, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = total_loss / len(train_loader)
    epoch_accuracy = correct / total
    
    # Print epoch statistics
    if epoch_accuracy > current_highest_accuracy:
        current_highest_accuracy = epoch_accuracy
        epochs_without_improvement = 0
        print(f"Epoch {epoch:<4} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f}")
    else:
        epochs_without_improvement += 1
        print(f"Epoch {epoch:<4} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f}", end="\r")
    
    if epochs_without_improvement >= patience:
        print(f"\nEarly stopping triggered at {epoch} epochs - No improvement in model accuracy in {patience} epochs")
        break

Epoch 1    - Loss: 0.4026 - Accuracy: 0.7992
Epoch 2    - Loss: 0.2795 - Accuracy: 0.8792
Epoch 3    - Loss: 0.2703 - Accuracy: 0.8853
Epoch 4    - Loss: 0.2595 - Accuracy: 0.8904
Epoch 5    - Loss: 0.2522 - Accuracy: 0.8945
Epoch 7    - Loss: 0.2437 - Accuracy: 0.8977
Epoch 10   - Loss: 0.2320 - Accuracy: 0.9043
Epoch 11   - Loss: 0.2321 - Accuracy: 0.9047
Epoch 12   - Loss: 0.2248 - Accuracy: 0.9067
Epoch 14   - Loss: 0.2195 - Accuracy: 0.9079
Epoch 16   - Loss: 0.2125 - Accuracy: 0.9105
Epoch 17   - Loss: 0.2102 - Accuracy: 0.9117
Epoch 20   - Loss: 0.2083 - Accuracy: 0.9119
Epoch 21   - Loss: 0.2054 - Accuracy: 0.9131
Epoch 22   - Loss: 0.2006 - Accuracy: 0.9170
Epoch 25   - Loss: 0.1952 - Accuracy: 0.9192
Epoch 27   - Loss: 0.1941 - Accuracy: 0.9204
Epoch 29   - Loss: 0.1849 - Accuracy: 0.9245
Epoch 34   - Loss: 0.1788 - Accuracy: 0.9264
Epoch 35   - Loss: 0.1787 - Accuracy: 0.9272
Epoch 36   - Loss: 0.1759 - Accuracy: 0.9291
Epoch 40   - Loss: 0.1693 - Accuracy: 0.9309
Epoch 42  

KeyboardInterrupt: 

In [24]:
torch.save(model.state_dict(), '../models/binary/hybrid_cnn_gan.pth')

### G. Evaluating both Models

In [25]:
model = HybridCNNGAN(input_size, output_size).to(device)
model.load_state_dict(torch.load('../models/binary/hybrid_cnn_gan.pth'))
model.eval()

C:\Users\Harman\AppData\Local\Temp\ipykernel_984\4028187911.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../models/binary/hybrid_cnn

HybridCNNGAN(
  (feature_extractor): CNNFeatureExtractor(
    (conv1): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (relu): ReLU()
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc): Linear(in_features=1248, out_features=64, bias=True)
  )
  (classifier): Linear(in_features=64, out_features=2, bias=True)
  (generator): Generator(
    (model): Sequential(
      (0): Linear(in_features=32, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=78, bias=True)
      (3): Tanh()
    )
  )
  (discriminator): Discriminator(
    (model): Sequential(
      (0): Linear(in_features=78, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=1, bias=True)
      (3): Sigmoid()
    )
  )
)

In [28]:
# Evaluation
from sklearn.metrics import accuracy_score, f1_score, classification_report

device = torch.device("cpu")
model.to(device)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)


def evaluate_model(model, X, y):
    with torch.no_grad():
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)
    acc = accuracy_score(y.cpu().numpy(), predicted.cpu().numpy())
    f1 = f1_score(y.cpu().numpy(), predicted.cpu().numpy(), average='weighted')
    report = classification_report(y.cpu().numpy(), predicted.cpu().numpy(), target_names=["BENIGN", "Malicious"])
    return acc, f1, report

model_acc, model_f1, model_report = evaluate_model(model, X_test_tensor, y_test_tensor)

print(f"Accuracy: {model_acc:.5f}\nF1 Score: {model_f1:.5f}\n\n{model_report}")

Accuracy: 0.98199
F1 Score: 0.98198

              precision    recall  f1-score   support

      BENIGN       0.99      0.97      0.98      1985
   Malicious       0.97      0.99      0.98      2012

    accuracy                           0.98      3997
   macro avg       0.98      0.98      0.98      3997
weighted avg       0.98      0.98      0.98      3997

